# Compute SGP4 Satellite Propagator Output in GCRS with `astropy`

This notebook will describe how to produce satellite position and velocity output from the SGP4 propagator using `python-sgp4` and how to convert them from their custom-defined frame to better defined frames using `astropy`.

Many amateur satellite applications use the legacy SGP4 Propagator to propagate the orbits that are distributed in [Two-Line-Elements (TLEs)](https://en.wikipedia.org/wiki/Two-line_element_set) that look like this:

>1 00005U 58002B   00179.78495062  .00000023  00000-0  28098-4 0  4753   
>2 00005  34.2682 348.7242 1859667 331.7664  19.3264 10.82419157413667

While this looks like some arcane cypher, you can read the orbital elements if you know where to look. Fortunately there are libraries in many languages to read this and output position and velocity in cartesian coordinates. Probably the most well known in Python is `python-sgp4` which you can find [here](https://github.com/brandon-rhodes/python-sgp4). This is based on the standard implementation documented in: 
>Vallado, David A., Paul Crawford, Richard Hujsak, and T.S. Kelso, “Revisiting Spacetrack Report #3,” presented at the >AIAA/AAS Astrodynamics Specialist Conference, Keystone, CO, 2006 August 21–24.

Ample information regarding the origins of the software, its mysteries and the "standard" software can be found at [Celestrak website](https://celestrak.com/publications/AIAA/2006-6753/). 

The problem that many people don't quite realise is that the output of the SGP4 is the positions and velocities in an "inertial-like" frame. Furthermore, it is poorly defined. But, by far the most common mistake is to use it _as if_ this is a proper "[Earth Centred Inertial Frame](https://en.wikipedia.org/wiki/Earth-centered_inertial)", like the Geocentric Celestial Reference Frame.

The rotation steps from TEME to the Earth based, rotating International Terrestrial Reference System (ITRS) are:
1. TEME to TIRS
2. TIRS to ITRS

For those who would like to plot the satellites on a Latitude, Longitude, Altitude setting or to analyse the visibility from the ground, ITRS is the natural stop. For those who require plotting satellite orbits on a globe or analysing them for orbital mechanics calculations, a final conversion from ITRS to GCRS is required. While TEME and TIRS will be defined here as custom frames, `astropy` provides the ITRS frame and the inertial GCRS frames as well as rotations between them. 

The first step will be to define the SGP4 output frame, called (True Equator, Mean Equinox) TEME and the intermediate frame Terrestrial Intermediate Reference System (TIRS) (which, we will assume to be equivalent to the older, equinox based Pseudo Earth Fixed Frame (PEF))[^1].

[^1]: For the pedantics out there, a reference system and a refernce frame are not the same thing, though I have mixed and matched a little bit in my text. Suffice it to say that a reference _frame_ is the realisation of the reference _system_. A wider explanation is beyond the scope of this practical coding example.

In [19]:
import astropy.coordinates.representation as r
from astropy.coordinates import BaseCoordinateFrame, TimeAttribute, frame_transform_graph, ITRS, \
    DynamicMatrixTransform
from astropy.coordinates.builtin_frames.utils import DEFAULT_OBSTIME, get_polar_motion, get_jd12
from astropy import _erfa as erfa


class TIRS(BaseCoordinateFrame):

    default_representation = r.CartesianRepresentation
    default_differential = r.CartesianDifferential

    obstime = TimeAttribute(default=DEFAULT_OBSTIME)


def _polar_mot_matrix(obstime):
    """
    Form the matrix of polar motion for a given date, IAU 2000.

    The matrix operates in the sense V(TRS) = rpom * V(CIP), meaning that it is the final rotation when computing the
    pointing direction to a celestial source.

    Parameters
    ----------
    obstime : Time
        time at which the polar motion should be calculated.
    Returns
    -------
        3x3 rotation matrix due to polar motion
    """
    # compute the polar motion p-matrix
    xp, yp = get_polar_motion(obstime)
    sp = erfa.sp00(*get_jd12(obstime, 'tt'))
    polar_mot_mat = erfa.pom00(xp, yp, sp)

    return polar_mot_mat


@frame_transform_graph.transform(DynamicMatrixTransform, TIRS, ITRS)
def tirs_to_itrs(tirs_coord, itrs_frame):

    tirs_to_itrs_mat = _polar_mot_matrix(tirs_coord.obstime)

    return tirs_to_itrs_mat


@frame_transform_graph.transform(DynamicMatrixTransform, ITRS, TIRS)
def itrs_to_tirs(itrs_coord, tirs_frame):

    itrs_to_tirs_mat = _polar_mot_matrix(itrs_coord.obstime).transpose()

    return itrs_to_tirs_mat

We start with the definition of the TIRS frame, which is a subclass of the `BaseCoordinateFrame` provided by `astropy`. The class has a default representation for position (in our case the standard (x,y,z) cartesian coordinates i.e., `CartesianRepresentation`) and for velocity (again, cartesian velocity i.e., `CartesianDifferential`). It also has a time (`obstime`) associated with each position and velocity - as the frames move and rotate with respect to each other at various speeds, this is the time where we freeze these frames and rotate between them.

The custom frame requires a definitions to convert into and out of this frame. While the top-level conversion is carried out by the `astropy` frame map, we indicate to `astropy` the definition of the particulars with the decorators (in this case TIRS to ITRS):
````
@frame_transform_graph.transform(DynamicMatrixTransform, TIRS, ITRS)
````
We also tell Astropy to use the output of this transformation as a Transformation Matrix that has to be computed at each time (a `DynamicMatrixTransform`) as opposed to a rotation offset that is computed once and reused again and again (a `StaticMatrixTransform`). 

The actual transformation is a rather simple rotation by the 3x3 Polar Motion Matrix that rotates position and velocity vectors. The inverse rotation is simply defined by the transpose. This `_polar_mot_matrix` (Polar Motion of the Earth at any given time) is computed by the underlying [ERFA](https://github.com/liberfa/erfa) routines, provided by `astropy`.

In [20]:
# Nominal mean angular velocity of the Earth [rad/s] as per GRS 80. (see IERS TN 36)
_w = np.array([0, 0, 7.292115E-5]) / u.s

class TEME(BaseCoordinateFrame):
    obstime = TimeAttribute(default=DEFAULT_OBSTIME)

    default_representation = r.CartesianRepresentation
    default_differential = r.CartesianDifferential


def _gmst82_angle(obstime):
    """
    Universal Time to Greenwich mean sidereal time (IAU 1982 model).

    Parameters
    ----------
    obstime : Time
        time at which the polar motion should be calculated.
    Returns
    -------
    float
        Greenwich mean sidereal time (radians)
    """
    # Get GMST82 angle in rad
    gmst82 = erfa.gmst82(*get_jd12(obstime, 'ut1')) * u.rad

    return gmst82


@frame_transform_graph.transform(FunctionTransform, TEME, TIRS)
def teme_to_tirs(teme_coord, tirs_frame):

    # TEME to TIRS basic rotation matrix
    teme_to_pef_mat = rotation_matrix(_gmst82_angle(teme_coord.obstime), axis='z')

    # rotate position vector: TEME to TIRS
    r_tirs = teme_coord.cartesian.transform(teme_to_pef_mat)

    # prepare rotation offset: w x r_TIRS
    wxr = CartesianRepresentation(_w).cross(r_tirs)

    # do the velocity rotation and then add rotation offset
    v_tirs = teme_coord.velocity.to_cartesian().transform(teme_to_pef_mat) - wxr
    v_tirs = CartesianDifferential.from_cartesian(v_tirs)

    # Prepare final coord vector with velocity
    tirs_coord = r_tirs.with_differentials(v_tirs)

    # Add coord data to the existing frame
    return tirs_frame.realize_frame(tirs_coord)


@frame_transform_graph.transform(FunctionTransform, TIRS, TEME)
def tirs_to_teme(tirs_coord, teme_frame):

    # TIRS to TEME basic rotation matrix
    teme_to_pef_mat = rotation_matrix(_gmst82_angle(tirs_coord.obstime), axis='z')
    pef_to_teme_mat = teme_to_pef_mat.transpose()

    # rotate position vector: TIRS to TEME
    r_teme = tirs_coord.cartesian.transform(pef_to_teme_mat)

    # prepare rotation offset: w x r_TIRS
    wxr = CartesianRepresentation(_w).cross(tirs_coord.cartesian)

    # add rotation offset and then do the velocity rotation
    v_teme = (tirs_coord.velocity.to_cartesian() + wxr).transform(pef_to_teme_mat)
    v_teme = CartesianDifferential.from_cartesian(v_teme)

    # Prepare final coord vector with velocity
    teme_coord = r_teme.with_differentials(v_teme)

    # Add coord data to the existing frame
    return teme_frame.realize_frame(teme_coord)

The second custom frame to define is the TEME frame itself. The general layout is similar to TIRS frame and, as the decoratior implies, the only way in and out of this frame is the neighbouring TIRS/PEF frame. The position transformation is a simple z axis rotation by an amount calculated by the Greenwich Mean Sidereal Time (essentially the angle defining where we are within the daily rotation of the Earth). However, the velocity rotation requires a correction due to the transformation between the "quasi-inertial" TEME frame and the rotating TIRS/PEF frame. The paper by Vallado et al. has detailed explanations into the mathematics and physics behind this.

Due to the more complicated nature of the TEME-TIRS conversions, as opposed to the previous TIRS-ITRS conversions above, doesn't enable us to tell Astropy to use a single rotation matrix. This time we tell Astropy that we will handle the entire rotation (`FunctionTransform`) and we finally return the full position and velocity, expressed in the target frame (`realize_frame()`).

One final remark is on the `_gmst82_angle()`, where we compute the actual z-axis rotation angle due to the daily rotation of the Earth that we described above. This is also computed by the underlying ERFA routines.

Now that the boring coordinate transformation classes are done, we can try how we to actually create the position and velocities and convert them to the more useful coordinate frames. For this, we are giong to use the example presented by D. Vallado in his paper above and also his book "Fundamentals of Astrodynamics and Applications 4th Ed." in Section 3.7, pp.233-234.

In [21]:
from sgp4.model import Satrec

from astropy.time import Time, TimeDelta
from astropy.coordinates import GCRS

# ****** Config and general setting up ******

# Init TLE
line1 = "1 00005U 58002B   00179.78495062  .00000023  00000-0  28098-4 0  4753"
line2 = "2 00005  34.2682 348.7242 1859667 331.7664  19.3264 10.82419157413667"

# Init satellite object from the TLE
sat = Satrec.twoline2rv(line1, line2)

# Define output time
output_time: Time = Time("2000:182", scale="utc", format="yday") + TimeDelta(0.78495062, format="jd")
output_time.format = "iso"

# SGP4 module requires time instances as jd and fraction
# This is compatible with Astropy time class
jd = output_time.jd1
frac = output_time.jd2

# Run the propagation and init pos and vel vectors in TEME
e, r_teme, v_teme = sat.sgp4(jd, frac)

We initialise the TLE simply as two strings and load this orbit data into the `Satrec` class. The instances of this object can output the position and velocity information at required time instances (in our case, it wiil be at `output_time`). The `output_time` is defined in a bit of a strange way, because in the example the target date is defined as day 182.78495062 of the year 2000. We immediately convert the format to a more human-readable ISO format.

The implementation of the legacy SGP4 algorithm requires the target time as two part Julian Dates (usually the date and the fraction parts), to conserve the precision in the old computers. Fortunately Astropy `Time` class supports this "two-part Julain Date" format with the `jd1` and `jd2` properties.

Finally we ask the `sat` object containing the TLE data to output the error, position and velocity at the `output_time`.

In [22]:
# Load the time, pos, vel info into astropy objects
coords = CartesianRepresentation(x=r_teme, unit=u.km, copy=True)\
    .with_differentials(CartesianDifferential(d_x=v_teme, unit=u.km / u.s, copy=True))

print(f"Time              : {output_time.iso}")
print(f"Pos vector (TEME) : {coords}")
print(f"Vel vector (TEME) : {coords.differentials['s']}")

# Load coordinates into TEME object
coord_teme = TEME(coords, obstime=output_time, representation_type="cartesian", differential_type="cartesian")

print(f"TEME object: {coord_teme}")

# Vallado pg.234
v_TEME_true = CartesianDifferential(d_x=[-2.232832783, -4.110453490, -3.157345433], unit=u.km / u.s, copy=True)
r_TEME_true = CartesianRepresentation(x=[-9060.47373569, 4658.70952502, 813.68673153], unit=u.km, copy=True)
r_TEME_true = TEME(r_TEME_true.with_differentials(v_TEME_true), obstime=output_time, representation_type="cartesian",
                   differential_type="cartesian")
# check the SGP4 results
print(f"r TEME diff      :  {(coord_teme.cartesian.without_differentials()  - r_TEME_true.cartesian.without_differentials()).norm().to(u.mm)}")
print(f"v TEME diff      :  {(coord_teme.velocity - r_TEME_true.velocity).norm().to(u.mm / u.s)}")

Time              : 2000-06-30 18:50:19.734
Pos vector (TEME) : (-9060.47373008, 4658.70953536, 813.68673948) km
Vel vector (TEME) : (-2.23283279, -4.11045349, -3.15734543) km / s
TEME object: <TEME Coordinate (obstime=2000-06-30 18:50:19.734): (x, y, z) in km
    (-9060.47373008, 4658.70953536, 813.68673948)
 (v_x, v_y, v_z) in km / s
    (-2.23283279, -4.11045349, -3.15734543)>
r TEME diff      :  14.20401675597324 mm
v TEME diff      :  0.00938637314497467 mm / s


As the outputs are in TEME, we will convert them into a `TEME` class instance and compare them against the "truth" values from the book. The resulting difference is only 14 mm in position and 0.009 mm/s in velocity. We are looking at a difference of only 14 mm at a distance of about 10000 km, which confirms that these two algorithms match very closely. This shouldn't be surprising, as `python-sgp4` uses this standard implementation of the algoithm under the hood.

The next step is to carry out the coordinate transformation and confirm its accuracy.

In [23]:
# Convert to GCRS
coord_gcrs = coord_teme.transform_to(GCRS(obstime=output_time))

print(f"Pos vector (GCRS) : {coord_gcrs.cartesian}")
print(f"Vel vector (GCRS) : {coord_gcrs.velocity}")

# Vallado pg.234 - this is actually in J2000 but the difference is less than a meter
v_GCRS_true = CartesianDifferential(d_x=[-2.233348094, -4.110136162, -3.157394074], unit=u.km / u.s, copy=True)
r_GCRS_true = CartesianRepresentation(x=[-9059.9413786, 4659.6972000, 813.9588875], unit=u.km, copy=True)
r_GCRS_true = GCRS(r_GCRS_true.with_differentials(v_GCRS_true), obstime=output_time, representation_type="cartesian",
                   differential_type="cartesian")

# check the coord conversion results
print(f"r GCRS diff      :  {(coord_gcrs.cartesian.without_differentials()  - r_GCRS_true.cartesian.without_differentials()).norm().to(u.mm)}")
print(f"v GCRS diff      :  {(coord_gcrs.velocity - r_GCRS_true.velocity).norm().to(u.mm / u.s)}")

Pos vector (GCRS) : (-9059.9416018, 4659.6971062, 813.95694615) km
Vel vector (GCRS) : (-2.23334739, -4.11013617, -3.15739449) km / s
r GCRS diff      :  1956.3909814713934 mm
v GCRS diff      :  0.8156527858108906 mm / s


The very first line of code with the `transform_to()` function carries out the actual rotation from TEME to TIRS, then TIRS to ITRS and finally from ITRS to GCRS. The top-level logic of this rotation is defined by Astropy and the underlying rotation algorithms are defined in our custom `TEME` and `TIRS` frame definition classes.

The difference between the values in the book and our algorithm is about 2 meters in position and 0.8 mm/s in velocity. This difference is explained by:
1. the difference in the rotation algorithms: Vallado uses the old FK5/IAU76 rotation and computes the value going through Nutation and Precession transformations. We go the opposite way, convert to ITRS and then use a much more modern conversion from ITRS to GCRS, provided by Astropy. 
2. the difference in the Earth Orientation Parameters that go into these frame transformation algorithms. 
3. the difference between the J2000 final frame in the book and GCRS in our example: J2000 and GCRS frames are very similar but are oriented slightly diffrently. J2000 is not offered by Astropy so we assumed the J2000 results in the book can be approximated as GCRS. About 0.3 m of the position difference is due to this approximation.

There are a couple of notes to add here. First, this example is primarily to show how a custom frame can be quickly defined in Astropy. While the example does work, a proper code would need to have some safety mechanisms for when the velocity vector is not provided or perhaps when the accelerations _are_ provided. Second, the "self-transform" functionality (to transform the coordinate to a different time in the same frame) found in other Astropy frames is missing.

Finally this example code can be found [here in Github](https://github.com/egemenimre/satstuff/blob/master/examples/sgp4/sgp4_teme_gcrs.py) and the custom frame definitions are [here for TEME](https://github.com/egemenimre/satstuff/blob/master/src/frames/teme.py) and [there for TIRS](https://github.com/egemenimre/satstuff/blob/master/src/frames/tirs.py).